In [1]:
# Also get the latest nightly Unsloth!
! pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 


fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", 
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",          
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            
] 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    
    use_gradient_checkpointing = "unsloth",
    use_rslora = False, 
    loftq_config = None, 
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token 
def formatting_prompts_func(examples):
    instructions = "Given an email, you have to classify the type of email. Each email belongs to one of the following categories: 0: Student inquiries /  1: Academic collaboration / 2: Corporate inquiries / 3: Research-related queries"
    inputs       = examples["email_content"]
    outputs      = examples["label"]
    texts = []
    for input, output in zip(inputs, outputs):

        text = alpaca_prompt.format("Given an email, you have to classify the type of email. Each email belongs to one of the following categories: 0: Student inquiries /  1: Academic collaboration inquiries / 2: Corporate inquiries", input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("csv",data_files="/content/new_email_data.csv", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
       
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:

FastLanguageModel.for_inference(model) 
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given an email, you have to classify the type of email. Each email belongs to one of the following categories: Student inquiries / Academic collaboration inquiries /Corporate inquiries", 
        "Dear Dr. White,I am writing to ask about the prerequisites for the advanced computer science course next semester. I want to ensure I meet all the requirements before registering.Thank you!Best,Oliver Johnson",
        "", 
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [36]:
import pandas as pd

test_data = pd.read_csv('test_email.csv')  
emails = test_data['email_content'].tolist()  
actual_labels = test_data['label'].tolist()    

In [ ]:
from transformers import TextStreamer


predicted_labels = []
FastLanguageModel.for_inference(model)

for email in emails:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                "Given an email, you have to classify the type of email. Each email belongs to one of the following categories: Student inquiries / Academic collaboration inquiries / Corporate inquiries",  # instruction
                email,  
                "",     
            )
        ], return_tensors="pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)

    generated = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

    generated_text = tokenizer.decode(generated[0], skip_special_tokens=True).strip()

    predicted_label = generated_text.split('\n')[-1].strip()  
    predicted_labels.append(predicted_label)


### Pushing Model to HuggingFace

In [ ]:

model.push_to_hub("your_name/lora_model", token = "...") 
tokenizer.push_to_hub("your_name/lora_model", token = "...") 